<a href="https://colab.research.google.com/github/manthanthakker/AI/blob/master/AudioClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
model_checkpoint = "facebook/wav2vec2-base"
batch_size = 32

In [63]:
# %%capture
# !pip install datasets==1.14
# !pip install transformers==4.11.3
# !pip install librosa

In [64]:
# from huggingface_hub import notebook_login

# notebook_login()

In [65]:
# %%capture
# !apt install git-lfs

# Step 1: Hugging Face Standard Dataset 

In [66]:
from datasets import load_dataset, load_metric
metric = load_metric("accuracy")

In [67]:
# dataset = load_dataset("superb", "ks")
# metric = load_metric("accuracy")

In [68]:
# dataset

In [69]:
# dataset["train"][1]["label"]

In [70]:
# train_data[1]

In [71]:
# labels = ["0","1","2","3","4","5","6","7","8","9"]
# label2id, id2label = dict(), dict()
# for i, label in enumerate(labels):
#     label2id[label] = str(i)
#     id2label[str(i)] = label

# id2label["8"]

In [72]:
# import random
# from IPython.display import Audio, display

# for _ in range(5):
#     rand_idx = random.randint(0, len(dataset["train"])-1)
#     example = dataset["train"][rand_idx]
#     audio = example["audio"]

#     print(f'Label: {id2label[str(example["label"])]}')
#     print(f'Shape: {audio["array"].shape}, sampling rate: {audio["sampling_rate"]}')
#     display(Audio(audio["array"], rate=audio["sampling_rate"]))
#     print()

# Step 1: Multi-lingual Audio (Gujarati digits)

In [73]:
# !git clone https://github.com/Nikunj1729/free-spoken-gujarati-digit-dataset
# !mv free-spoken-gujarati-digit-dataset load_guj

In [74]:
# train_data.cleanup_cache_files()
# val_data.cleanup_cache_files()
# test_data.cleanup_cache_files()
# dataset=None
# train_data=None
# test_data=None

In [75]:
import glob
all_wavs=glob.glob("/content/load_guj/**/*.wav", recursive=True)
len(all_wavs)
from sklearn.model_selection import train_test_split
x, Y=train_test_split(all_wavs, test_size=0.2)
print(len(x))
print(len(Y))
import json
#train_file = open("/content/load_guj/train.jsonl", "w")
#n = train_file.write(json.dumps(x))
#train_file.close()

import json

count=0
with open('/content/load_guj/train.jsonl', 'w') as outfile:
    for entry in x:
        json.dump(entry, outfile)
        outfile.write('\n')
        count=count+1
        if(count>50):
          break

with open("/content/load_guj/test.jsonl", "w") as outfile:
    for entry in Y:
        json.dump(entry, outfile)
        outfile.write('\n')
!cp /content/load_guj/test.jsonl /content/load_guj/dev.jsonl

1552
388


In [76]:
from datasets import load_dataset, load_metric
train_data=load_dataset("/content/load_guj/load_guj.py",split="train")
val_data=load_dataset("/content/load_guj/load_guj.py",split="test")
test_data=val_data

No config specified, defaulting to: new_dataset/train
Reusing dataset new_dataset (/root/.cache/huggingface/datasets/new_dataset/train/1.1.0/cec7da7558d173c9ee688d48951004f9959b1dca0a1ad1890a194efe0e260c1f)
No config specified, defaulting to: new_dataset/train
Reusing dataset new_dataset (/root/.cache/huggingface/datasets/new_dataset/train/1.1.0/cec7da7558d173c9ee688d48951004f9959b1dca0a1ad1890a194efe0e260c1f)


In [77]:
from datasets.dataset_dict import DatasetDict
from datasets.dataset_dict import DatasetDict
dataset={}
dataset["train"]=train_data
dataset["test"]=test_data
dataset=DatasetDict(dataset)
# dataset["train"]
# dataset["train"][1]["audio"]
# dataset["train"][1]["label"]
# dataset["train"].features["label"]
# dataset["train"]
# dataset["train"][2]

In [78]:
labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

id2label["9"]

'9'

In [79]:
label2id

{'0': '0',
 '1': '1',
 '2': '2',
 '3': '3',
 '4': '4',
 '5': '5',
 '6': '6',
 '7': '7',
 '8': '8',
 '9': '9',
 '_silence_': '10',
 '_unknown_': '11'}

In [80]:
import random
from IPython.display import Audio, display

for _ in range(5):
    rand_idx = random.randint(0, len(dataset["train"])-1)
    example = dataset["train"][rand_idx]
    audio = example["audio"]

    print(f'Label: {id2label[str(example["label"])]}')
    print(f'Shape: {audio["array"].shape}, sampling rate: {audio["sampling_rate"]}')
    display(Audio(audio["array"], rate=audio["sampling_rate"]))
    print()

Label: 6
Shape: (12260,), sampling rate: 16000



Label: 2
Shape: (10747,), sampling rate: 16000



Label: 1
Shape: (11223,), sampling rate: 16000



Label: 6
Shape: (11330,), sampling rate: 16000



Label: 1
Shape: (9953,), sampling rate: 16000


In [81]:
dataset["train"][2]

{'audio': {'array': array([-2.6911581e-05, -4.7613788e-04, -9.2794903e-04, ...,
         -1.9840747e-03, -1.1656560e-03,  0.0000000e+00], dtype=float32),
  'path': '/content/load_guj/R4 - Saurashtra/S4/R4S4T3D8.wav',
  'sampling_rate': 16000},
 'file': '/content/load_guj/R4 - Saurashtra/S4/R4S4T3D8.wav',
 'label': 8}

# Step 2: Pre processing

In [82]:
max_duration = 1.0  # seconds

In [83]:
from transformers import AutoFeatureExtractor
model_checkpoint = "facebook/wav2vec2-base"
batch_size = 32
feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)
feature_extractor

loading configuration file https://huggingface.co/facebook/wav2vec2-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c7746642f045322fd01afa31271dd490e677ea11999e68660a92619ec7c892b4.ce1f96bfaf3d7475cb8187b9668c7f19437ade45fb9ceb78d2b06a2cec198015
/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Model config Wav2Vec2Config {
  "activation_dropout": 0.0,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
 

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

In [84]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, 
        sampling_rate=feature_extractor.sampling_rate, 
        max_length=int(feature_extractor.sampling_rate * max_duration), 
        truncation=True, 
    )
    return inputs

In [85]:
preprocess_function(dataset['train'][:5])

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:158: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


{'input_values': [array([-0.00343115,  0.0268481 ,  0.06603188, ..., -0.003115  ,
        0.00139647,  0.00493543], dtype=float32), array([-1.2691899e-03, -9.7545749e-04,  3.9509422e-04, ...,
        5.8767633e-03,  4.5574945e-03, -3.3009044e-05], dtype=float32), array([-2.9383608e-04, -6.2776478e-03, -1.2295891e-02, ...,
       -2.6363762e-02, -1.5462211e-02,  6.4633219e-05], dtype=float32), array([ 0.1360805 ,  0.23976848,  0.23395076, ..., -0.05796618,
       -0.03984864,  0.00543909], dtype=float32), array([-0.00322145,  0.03012054,  0.07010429, ..., -0.08256871,
       -0.05468863, -0.00229904], dtype=float32)]}

In [86]:
encoded_dataset = dataset.map(preprocess_function, remove_columns=["audio", "file"], batched=True)
encoded_dataset

Loading cached processed dataset at /root/.cache/huggingface/datasets/new_dataset/train/1.1.0/cec7da7558d173c9ee688d48951004f9959b1dca0a1ad1890a194efe0e260c1f/cache-6ad58047d24d509a.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/new_dataset/train/1.1.0/cec7da7558d173c9ee688d48951004f9959b1dca0a1ad1890a194efe0e260c1f/cache-a257493ec7cc09c2.arrow


DatasetDict({
    train: Dataset({
        features: ['input_values', 'label'],
        num_rows: 51
    })
    test: Dataset({
        features: ['input_values', 'label'],
        num_rows: 388
    })
})

# Training/Finetuning

In [87]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    model_checkpoint, 
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

loading configuration file https://huggingface.co/facebook/wav2vec2-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c7746642f045322fd01afa31271dd490e677ea11999e68660a92619ec7c892b4.ce1f96bfaf3d7475cb8187b9668c7f19437ade45fb9ceb78d2b06a2cec198015
/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Model config Wav2Vec2Config {
  "activation_dropout": 0.0,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
 

In [88]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-ks",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [89]:
import numpy as np

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [90]:
dataset

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'label'],
        num_rows: 51
    })
    test: Dataset({
        features: ['file', 'audio', 'label'],
        num_rows: 388
    })
})

In [91]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

In [92]:
encoded_dataset["train"][1]

{'input_values': [-0.0012691898737102747,
  -0.0009754574857652187,
  0.0003950942191295326,
  0.0013603270053863525,
  0.002227536868304014,
  0.0034051297698169947,
  0.004305865615606308,
  0.005250157788395882,
  0.0064741214737296104,
  0.006888297852128744,
  0.007782679982483387,
  0.00817088596522808,
  0.008522817865014076,
  0.008699683472514153,
  0.009106600657105446,
  0.009633886627852917,
  0.009820875711739063,
  0.010050726123154163,
  0.009688937105238438,
  0.009452766738831997,
  0.008636255748569965,
  0.008035107515752316,
  0.007544823456555605,
  0.0071237520314753056,
  0.007159009110182524,
  0.006100732367485762,
  0.0059182606637477875,
  0.005541485734283924,
  0.005000593606382608,
  0.0047367047518491745,
  0.003454019548371434,
  0.002940204693004489,
  0.0023258309811353683,
  0.0014021276729181409,
  0.001199354068376124,
  0.0007144011906348169,
  0.00023908255388960242,
  -0.0005572452791966498,
  -0.0013151647290214896,
  -0.0019140699878335,
  -0.0

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 51
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 4
  Total optimization steps = 5


In [ ]:
trainer.push_to_hub()

In [ ]:
trainer.evaluate()